In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
def process_image(image_path, IMG_SIZE=224):

    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, channels=3, expand_animations= False)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    return image

In [ ]:
images_loc = []
labels = []
for i in os.listdir("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/nofire"):
    images_loc.append(process_image("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/nofire"+"/"+i))
    labels.append(0)
for i in os.listdir("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/fire"):
    images_loc.append(process_image("/kaggle/input/forest-fire-dataset/Forest Fire Dataset/Training/fire"+"/"+i))
    labels.append(1)

In [ ]:
classnames = np.array(["nofire", "fire"])

In [ ]:
images_loc = np.array(images_loc)
labels = np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_testy, y_train, y_testy = train_test_split(images_loc, labels, test_size = 0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_testy, y_testy, test_size = 0.5)

In [ ]:
len(X_test), len(y_test)

In [ ]:
import matplotlib.pyplot as plt
def show_25_images(images, labels):
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax=plt.subplot(5,5,i+1)
        plt.imshow(images[i])
        plt.title(classnames[labels[i]])
        plt.axis("off")

In [ ]:
show_25_images(X_train, y_train)

## Using Custom cnn

In [ ]:
import tensorflow_hub as hub
model = tf.keras.Sequential([
   hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/100-224-classification/versions/2",
               trainable=True, arguments=dict(batch_norm_momentum=0.997))
])
model.add(tf.keras.layers.Dense(6, activation='softmax'))
model.build([None, 224, 224, 3]) 

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_mobile_netv2.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x = X_train, 
          y = y_train,
          epochs=25,
          validation_data = (X_val, y_val),
         callbacks = [checkpoint_callback])

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
img_path = input("Enter the image path for prediction : ")
image = process_image(img_path) 
image = np.expand_dims(image, axis=0) 
p = np.argmax(model.predict(image))
a = plt.imread(img_path)
plt.title(f"Prediction : {classnames[p]}")
plt.imshow(a)

In [ ]:
model.save("best_mobile_netv2_2.keras")